In [1]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [2]:
import numpy as np 
import pandas as pd 

import math
import random 
import os 
from albumentations.pytorch.transforms import ToTensorV2
import pickle
get_cpkl = exec
import torch 
import cv2
import timm

from tqdm import tqdm 
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
import albumentations as A 

from torch.utils.data import Dataset 
from torch import nn
import torch.nn.functional as F 
import dill as pickle5
import gc
import cupy
import cudf
import cuml


In [3]:
class CFG:
    img_size = 512
    batch_size = 12
    seed = 42
    
    device = 'cuda'
    classes = 11014
    
    img_size1 = 640
    model_name1 = 'eca_nfnet_l1'
    model_path1 = '../input/fdgfdgdfgdfg/EfficientNetB3_NF_M0.7_512_42.h5'
    
    img_size2 = 384
    model_name2 = 'efficientnet_b3'
    model_path2 = '../input/fdgfdgdfgdfg/efficientnet-1.1.0-SW-py3-none-any.whl'
    
    scale = 30 
    margin = 0.5

In [4]:
df = pd.read_csv('../input/shopee-product-matching/test.csv')
# df = pd.read_csv('../input/shopee-product-matching/train.csv')
# df = pd.concat([df,df,df[:2000]]).reset_index(drop=True)
# image_paths = '../input/shopee-product-matching/train_images/' + df['image']
image_paths = '../input/shopee-product-matching/test_images/' + df['image']

df_cu = cudf.DataFrame(df)

In [5]:
df

,posting_id,image,image_phash,title
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng


In [6]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(CFG.seed)

In [7]:
def masked_categorical_crossentropy(output,target):
    y_true_masked = target[target!= -100]
    y_pred_masked = output[target!= -100]
    loss =  nn.CrossEntropyLoss()(y_pred_masked,y_true_masked)
    return loss

In [8]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions1'], row['image_predictions2'], row['text_predictions'], row['phash_predictions']])
    return ' '.join( np.unique(x))

In [9]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [10]:
get_image_predictions = pickle5.load(open("../input/fdgfdgdfgdfg/f_image_p.pkl","rb"))

In [11]:
def get_test_transforms1():

    return A.Compose(
        [
            A.Resize(CFG.img_size1,CFG.img_size1,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

In [12]:
def get_test_transforms2():

    return A.Compose(
        [
            A.Resize(CFG.img_size2,CFG.img_size2,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

In [13]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [14]:
def Gen_color_Img(S, names):
    Images = []
    for index in range(S):
        # Read the raw data
        IMG = tf.io.read_file(names[index])
        # Decoding the raw data
        IMG = tf.io.decode_png(IMG, channels=1)
        # Resize the image
        IMG = tf.image.resize(IMG, [2048,2048],method='bicubic')
        Images.append(IMG)
    return Images

In [15]:
class ShopeeDataset(Dataset):
    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

In [16]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output

get_cpkl(pickle.load(open("../input/fdgfdgdfgdfg/c_ShopeeModel.pkl","rb")))

In [17]:
def new_parrall(model, model_path):
    state = torch.load(model_path, map_location='cuda:0')
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state.items():
        k=k[7:]
        new_state_dict[k]=v
    model.load_state_dict(new_state_dict)
    return model 

In [18]:
def get_image_embeddings(image_paths, model_name, model_path, get_test_transforms):
    embeds = []
    model_arch = model_name
    model = ShopeeModel(11014, model_name, False, False)
    model.eval()
    

    model = new_parrall(model, model_path)
    model = model.to(CFG.device)
    

    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [19]:
def train_fn(dataloader,model,optimizer,device,scheduler,epoch):
    model.train()
    loss_score = AverageMeter()
    
    tk0 = tqdm(enumerate(dataloader), total=len(dataloader))
    for bi,d in tk0:
        
        batch_size = d[0].shape[0]

        input_ids = d[0]
        targets = d[1]

        input_ids = input_ids.to(device,dtype=torch.long)
        targets = targets.to(device)

        optimizer.zero_grad()

        output = model(input_ids=input_ids,labels=targets)
        
        loss = output.loss       
        
        loss.backward()
        optimizer.step()
        
        loss_score.update(loss.detach().item(), batch_size)
        tk0.set_postfix(Train_Loss=loss_score.avg,Epoch=epoch,LR=optimizer.param_groups[0]['lr'])
        
        if scheduler is not None:
                scheduler.step()
        
    return loss_score

In [20]:
def get_text_predictions(df, max_features = 25_000):
    
    model = TfidfVectorizer(stop_words = 'english', binary = True, max_features = max_features)
    text_embeddings = model.fit_transform(df_cu['title']).toarray()
    preds = []
    CHUNK = 1024*4

    print('Finding similar titles...')
    CTS = len(df)//CHUNK
    if len(df)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b,len(df))
        print('chunk',a,'to',b)

        # COSINE SIMILARITY DISTANCE
        cts = cupy.matmul( text_embeddings, text_embeddings[a:b].T).T

        for k in range(b-a):
            IDX = cupy.where(cts[k,]>0.75)[0]
            o = df.iloc[cupy.asnumpy(IDX)].posting_id.values
            preds.append(o)
    
    del model,text_embeddings
    gc.collect()
    return preds

In [21]:
def run():
    # Defining DataSet
    train_dataset = ShopeeDataset(
        csv=data
    )
        
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        pin_memory=True,
        drop_last=True,
        num_workers=NUM_WORKERS
    )
    
    
    # Defining Model for specific fold
    model = AutoModelForMaskedLM.from_pretrained(transformer_model)
    print(model)
    model.to(device)

        
    # Defining Optimizer with weight decay to params other than bias and layer norms
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
            ]  
    
    optimizer = AdamW(optimizer_parameters, lr=LR)
    
    #Defining LR SCheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=len(train_loader)*5, 
        num_training_steps=len(train_loader)*EPOCHS
    )
        
    # THE ENGINE LOOP
    for epoch in range(EPOCHS):
        train_loss = train_fn(train_loader, model,optimizer, device,scheduler=scheduler,epoch=epoch)
          
    model.save_pretrained('./')

In [22]:
model_arch = CFG.model_name1
image_embeddings1 = get_image_embeddings(image_paths.values, model_arch, CFG.model_path1, get_test_transforms1)
torch.cuda.empty_cache()
 
model_arch = "swin_large_patch4_window12_384"
image_embeddings2 = get_image_embeddings(image_paths.values, model_arch, CFG.model_path2, get_test_transforms2)
torch.cuda.empty_cache()

3072


100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


Our image embeddings shape is (3, 3072)
1536


100%|██████████| 1/1 [00:00<00:00,  2.13it/s]


Our image embeddings shape is (3, 1536)


In [23]:
image_predictions1 = get_image_predictions(df, image_embeddings1, threshold = 0.7, threshold_a=0.2)
del image_embeddings1
gc.collect()

image_predictions2 = get_image_predictions(df, image_embeddings2, threshold = 0.5, threshold_a=0.2)
del  image_embeddings2
gc.collect()

torch.cuda.empty_cache()

text_predictions = get_text_predictions(df, max_features = 25_000)

100%|██████████| 3/3 [00:00<00:00, 4198.50it/s]


Finding similar titles...
chunk 0 to 3


In [24]:
df['image_predictions1'] = image_predictions1
df['image_predictions2'] = image_predictions2
df['text_predictions'] = text_predictions
df['phash_predictions'] = df.image_phash.map(df.groupby('image_phash').posting_id.agg('unique').to_dict())
df['matches'] = df.apply(combine_predictions, axis = 1)
df[['posting_id', 'matches']].to_csv('submission.csv', index = False)